In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import os, pathlib, glob, random
import numpy as np
import matplotlib.pyplot as plt 

from sklearn.metrics import confusion_matrix
import scipy
from scipy import io

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 32
output_nodes = 8
learning_rate = 0.01

In [3]:
test_data_path_mw =r"C:\Users\admin\Desktop\ARTH SHAH\Emotion Challange Dataset\LFCC\Val"

In [4]:
test_data_path_ms=r"C:\Users\admin\Desktop\ARTH SHAH\Emotion Challange Dataset\LP_EMO_Challenge\LP_12\Val"

In [5]:
from PIL import Image

class PtDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        self.files = []
        self.transform = transform
        
        for c in self.classes:
            c_dir = os.path.join(directory, c)
            c_files = [(os.path.join(c_dir, f), self.class_to_idx[c]) for f in os.listdir(c_dir) if f.endswith('.jpg')]
            self.files.extend(c_files)
        random.shuffle(self.files)
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        filepath, label = self.files[idx]
        try:
            data = Image.open(filepath).convert('RGB')
            
            # Apply transformations if any
            if self.transform:
                data = self.transform(data)
                
        except Exception as e:
            print(f"Error loading file {filepath}: {str(e)}")
            return None
        
        return data, label


In [6]:
from torchvision import transforms

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the images
])

transformed_test_dataset_mw = PtDataset(test_data_path_mw, transform=transform)
transformed_test_dataset_ms = PtDataset(test_data_path_ms, transform=transform)

In [7]:
test_dataloader_mw = DataLoader(transformed_test_dataset_mw, batch_size=batch_size, shuffle=False)
test_dataloader_ms = DataLoader(transformed_test_dataset_ms, batch_size=batch_size, shuffle=False)

In [8]:
test_dataset_mw = PtDataset(test_data_path_mw)
test_dataset_ms = PtDataset(test_data_path_ms)

In [9]:
test_count_mw = len(test_dataset_mw)
test_count_ms = len(test_dataset_ms)
print(test_count_mw)
print(test_count_ms)

4634
4634


In [10]:
import torch.nn as nn

class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(2 * 128 * 128, num_classes)  # Adjusted input size after convolutions and pooling

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
num_classes = 2
cnn_model = CNNClassifier(num_classes)

In [11]:
import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)

In [12]:
print(cnn_model)

CNNClassifier(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=32768, out_features=2, bias=True)
)


In [13]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import os, pathlib, glob, random
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import confusion_matrix
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
from transformers.models.whisper.modeling_whisper import WhisperModel, WhisperEncoder
from transformers.models.whisper.configuration_whisper import WhisperConfig
from typing import Optional, Tuple, Union
import torch
import librosa 
import matplotlib.pyplot as plt
import numpy as np
import os, glob, pickle
import scipy.io as sio
from tqdm import tqdm
import multiprocessing as mp 
import torch.optim as optim

In [14]:
model_mw_cnn= torch.load(r"/kaggle/input/mftyfjhv/code1_MW/best_accuracy_model_CNN.pth")
model_ms_cnn= torch.load(r"/kaggle/input/mftyfjhv/code2_MS/mel_spec_model_CNN.pth")

## Fusion

In [15]:
from sklearn.metrics import confusion_matrix,balanced_accuracy_score,accuracy_score,f1_score,jaccard_score,matthews_corrcoef,hamming_loss,roc_curve,accuracy_score

In [16]:
import numpy as np

def eer(confusion_matrix):
    '''
    Calculates the Equal Error Rate (EER) for multi-class classification using the confusion matrix.
   
    Parameters:
        confusion_matrix (array-like): Confusion matrix.
       
    Returns:
        eer (double): Equal Error Rate.
    '''
    n_classes = confusion_matrix.shape[0] # number of classes
    eer_list = [] # list to store EER for each class
   
    for i in range(n_classes):
        tp = confusion_matrix[i, i] # true positives for class i
        fn = np.sum(confusion_matrix[i, :]) - tp # false negatives for class i
        fp = np.sum(confusion_matrix[:, i]) - tp # false positives for class i
        tn = np.sum(confusion_matrix) - tp - fn - fp # true negatives for class i

        # Compute False Acceptance Rate (FAR) and False Rejection Rate (FRR)
        far = fp / (fp + tn)
        frr = fn / (tp + fn)
        
        # Find the point where the FAR and FRR curves intersect
        eer_list.append((far + frr) / 2)
       
    eer = np.sum(eer_list)/n_classes
    return eer

In [25]:
EER = 0
correct = 0
total = 0
y_true = []
y_pred = []
non_empty_batches = 0
last_EER = 0
with torch.no_grad():
    for batch_idx, (img_ms, label) in enumerate(test_dataloader_ms):
        if torch.cuda.is_available():
            img_ms = img_ms.cuda()
            label = label.cuda()

        optimizer.zero_grad()
        mfcc_outputs = model_ms_cnn(img_ms)
        mfcc_outputs = torch.softmax(mfcc_outputs,dim=1)
        eps = 1e-8
        _, predicted = torch.max(mfcc_outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        y_true.extend(label.tolist())
        y_pred.extend(predicted.tolist())

accuracy = 100 * correct / total
cm = confusion_matrix(y_true,y_pred)
EER = eer(cm)
print(f"Accuracy = {accuracy:.2f}%, EER = {EER:.4f}")

Accuracy = 95.08%, EER = 0.0498


In [18]:
model_mw_cnn.eval()
model_ms_cnn.eval()

CNNClassifier(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=32768, out_features=2, bias=True)
)

In [26]:
from sklearn.metrics import f1_score, jaccard_score, confusion_matrix, matthews_corrcoef, hamming_loss
alphas = np.linspace(0.1, 0.9, num=10)

for alpha in alphas:
    EER = 0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    non_empty_batches = 0
    last_EER = 0
    with torch.no_grad():
        
        for batch_idx, ((img_mw, label), (img_ms, label)) in enumerate(zip(test_dataloader_mw, test_dataloader_ms)):
    # Your code here
            ##feature_mfcc = feature_mfcc
            ##feature_lfrcc = feature_lfrcc
#ahi badlo 2 model
            if torch.cuda.is_available():
                        img_mw = img_mw.cuda()
                        label = label.cuda()

            optimizer.zero_grad()
            mfcc_outputs = model_mw_cnn(img_mw)
            if torch.cuda.is_available():
                        img_ms = img_ms.cuda()
                        label = label.cuda()

            optimizer.zero_grad()
            mfcc_outputs2 = model_ms_cnn(img_ms)
            # lfrcc_outputs = model_lfrcc_lstm(feature_mfcc)
            mfcc_outputs = torch.softmax(mfcc_outputs,dim=1)
            mfcc_outputs2 = torch.softmax(mfcc_outputs2,dim=1)
            # lfrcc_outputs = torch.softmax(lfrcc_outputs,dim=1)
            # fused_output = ( alpha * mfcc_outputs ) + ((1 - alpha) * lfrcc_outputs)
            fused_output = ( alpha * mfcc_outputs ) + ((1 - alpha) * mfcc_outputs2)
            eps = 1e-8
            _, predicted = torch.max(fused_output.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
            y_true.extend(label.tolist())
            y_pred.extend(predicted.tolist())
    old_acc = 100*(correct/total)
    accuracy = accuracy_score(y_true,y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true,y_pred)
    print(f"---------------Results for {alpha}-------------------")
    f1 = f1_score(y_true, y_pred, average='macro')
    jaccard = jaccard_score(y_true, y_pred, average='macro')
    cm = confusion_matrix(y_true, y_pred)
    mc = matthews_corrcoef(y_true,y_pred)
    hm = hamming_loss(y_true,y_pred)
    print(f"balanced accuracy = {balanced_accuracy*100:.2f}%, EER = {(eer(cm))*100:.4f}%")
    print(f"F1 Score = {f1:.4f}, Jaccard Score = {jaccard:.4f},Matthews Coeff = {mc:.4f}, Hamming Loss: {hm:.4f}")
    print(f"Confusion Matrix:")
    print(cm)
    print(f"-----------------------------------------------------\n")

---------------Results for 0.1-------------------
balanced accuracy = 94.89%, EER = 5.1106%
F1 Score = 0.9494, Jaccard Score = 0.9037,Matthews Coeff = 0.9000, Hamming Loss: 0.0505
Confusion Matrix:
[[2312   58]
 [ 176 2088]]
-----------------------------------------------------

---------------Results for 0.18888888888888888-------------------
balanced accuracy = 94.89%, EER = 5.1076%
F1 Score = 0.9494, Jaccard Score = 0.9037,Matthews Coeff = 0.8999, Hamming Loss: 0.0505
Confusion Matrix:
[[2309   61]
 [ 173 2091]]
-----------------------------------------------------

---------------Results for 0.2777777777777778-------------------
balanced accuracy = 94.89%, EER = 5.1066%
F1 Score = 0.9494, Jaccard Score = 0.9037,Matthews Coeff = 0.8999, Hamming Loss: 0.0505
Confusion Matrix:
[[2308   62]
 [ 172 2092]]
-----------------------------------------------------

---------------Results for 0.3666666666666667-------------------
balanced accuracy = 94.85%, EER = 5.1498%
F1 Score = 0.9490, Jac